In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics
from sklearn.metrics import make_scorer, accuracy_score
import json
import scipy.stats 
import pickle
import sklearn.model_selection
import sklearn.ensemble
import itertools
from multiprocessing import Pool
import time

In [2]:
#load files
results = pd.read_csv('../data/results/model_results.csv', index_col=0)

In [120]:
#build a random forest model to predict slow/fast codon labels based on position, structure (pLDDT), and amino acid
#this can be used as a comparison for how much of the model's gain in accuracy can be explained just by position and structure
aa_enc = sklearn.preprocessing.OneHotEncoder(sparse=False)
idx = ~results.pLDDT.isna()
aa_enc.fit(np.array(results.AA[idx]).reshape(-1, 1))
aa_onehot = aa_enc.transform(np.array(results.AA[idx]).reshape(-1, 1))
X = np.concatenate((aa_onehot, np.array(results[['pLDDT', 'position']].loc[idx])), axis=1)
y = np.array(results[['label']].loc[idx]).flatten()
weights = np.array(results['weight'][idx])

In [130]:
#implement grid search
def crossvalidate(clf, X, y, weights, folds=None, k=5, return_auc = False): #too include weights
    if folds is None:
        folds = np.random.random_integers(k, size=y.shape)
    else:
        assert(k == max(folds))
    scores = []
    auc_scores = []
    for i in range(1,k+1):
        test_idx = folds == i
        clf.fit(X[~test_idx], y[~test_idx], sample_weight=weights[~test_idx])
        scores.append(clf.score(X[test_idx], y[test_idx], sample_weight=weights[test_idx]))
        X_pred = clf.predict_proba(X[test_idx])[:,1]
        auc_scores.append(sklearn.metrics.roc_auc_score(y[test_idx], X_pred,sample_weight=weights[test_idx]))
    if return_auc:
        return scores, auc_scores
    else:
        return scores
    
def crossvalidate_worker(params):
    start_time = time.time()
    print('Start running ', str(params[0]))
    scores = crossvalidate(*params)
    print('Mean accuracy for', str(params[0]), 'is', np.mean(scores), '. Run finished after ', (time.time()-start_time)//60, 'minutes')
    return scores


def gridsearch(X, y, weights, folds, param_grid):
    keys = param_grid.keys()
    values = param_grid.values()
    param_combinations = [dict(zip(keys, combo)) for combo in list(itertools.product(*values))]
    clf_list = [sklearn.ensemble.RandomForestClassifier(**params) for params in param_combinations]
    with Pool(10) as pool:
        scores_list = pool.map(crossvalidate_worker, zip(clf_list, itertools.repeat(X), itertools.repeat(y), itertools.repeat(weights), itertools.repeat(folds)))
    mean_scores = [np.mean(scores) for scores in scores_list]
    print('\n\nBest score is', max(mean_scores), 'for params', param_combinations[mean_scores.index(max(mean_scores))])
    return {'clf_list':clf_list, 'scores_list':scores_list, 'param_combinations':param_combinations}

In [131]:
param_grid =  {'n_estimators': [20,100,500],
    'max_depth': [5, 10, 20],
    'min_samples_split': [50, 500, 5000],
    'bootstrap': [False]}

np.random.seed(12)
folds = np.random.random_integers(5, size=y.shape)
grid_scores = gridsearch(X, y, weights, folds, param_grid)

/mnt/lareaulab/shelen/anaconda3/envs/gen_env/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: This function is deprecated. Please call randint(1, 5 + 1) instead
  import sys


Start running  RandomForestClassifier(bootstrap=False, max_depth=5, min_samples_split=50,
                       n_estimators=20)
Start running  RandomForestClassifier(bootstrap=False, max_depth=5, min_samples_split=500,
                       n_estimators=20)
Start running  RandomForestClassifier(bootstrap=False, max_depth=5, min_samples_split=5000,
                       n_estimators=20)
Start running  RandomForestClassifier(bootstrap=False, max_depth=10, min_samples_split=50,
                       n_estimators=20)
Start running  RandomForestClassifier(bootstrap=False, max_depth=10, min_samples_split=500,
                       n_estimators=20)
Start running  RandomForestClassifier(bootstrap=False, max_depth=10, min_samples_split=5000,
                       n_estimators=20)
Start running  RandomForestClassifier(bootstrap=False, max_depth=20, min_samples_split=50,
                       n_estimators=20)
Start running  RandomForestClassifier(bootstrap=False, max_depth=20, min_samples

In [134]:
param_grid_bootstrap =  {'n_estimators': [20,100,500],
    'max_depth': [5, 10, 20],
    'min_samples_split': [10, 100, 1000],
    'bootstrap': [True],
    'max_samples' : [1000, 10000, 100000],
}
grid_scores_bootstrap = gridsearch(X, y, weights, folds, param_grid_bootstrap)

Start running  RandomForestClassifier(max_depth=5, max_samples=1000, min_samples_split=10,
                       n_estimators=20)
Start running  RandomForestClassifier(max_depth=5, max_samples=1000, min_samples_split=100,
                       n_estimators=20)
Start running  RandomForestClassifier(max_depth=5, max_samples=1000, min_samples_split=1000,
                       n_estimators=20)
Start running  RandomForestClassifier(max_depth=10, max_samples=1000, min_samples_split=10,
                       n_estimators=20)
Start running  RandomForestClassifier(max_depth=10, max_samples=1000, min_samples_split=100,
                       n_estimators=20)
Start running  RandomForestClassifier(max_depth=10, max_samples=1000, min_samples_split=1000,
                       n_estimators=20)
Start running  RandomForestClassifier(max_depth=20, max_samples=1000, min_samples_split=10,
                       n_estimators=20)
Start running  RandomForestClassifier(max_depth=20, max_samples=1000, min

In [135]:
#so, best score is 0.5179321285417593 for params {'n_estimators': 500, 'max_depth': 10, 'min_samples_split': 5000, 'bootstrap': False}